In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline


In [2]:
# Load pickled data
import pickle

# TODO: fill this in based on where you saved the training and testing data
training_file = "traffic-signs-data/train.p"
testing_file = "traffic-signs-data/test.p"

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [3]:
X_train.shape
print(X_train[0,:,:,:].shape)
print(len(X_train))

(32, 32, 3)
34799


In [4]:
from sklearn.cross_validation import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

/Users/Ryosuke/anaconda/envs/Udacity/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
features = tf.placeholder(tf.float32,[None,32,32,3])
labels = tf.placeholder(tf.int32)
labels_oh = tf.one_hot(labels,43)

In [6]:
def LeNet(x):
    conv1 = tf.layers.conv2d(x,6,2,padding="valid",activation=tf.nn.relu)
    max_pool1 = tf.layers.max_pooling2d(conv1,2,2,padding="valid")
    conv2 = tf.layers.conv2d(max_pool1,16,1,padding="valid",activation=tf.nn.relu)
    max_pool2 = tf.layers.max_pooling2d(conv2,2,2,padding ="valid")
    flatten = tf.contrib.layers.flatten(max_pool2)
    dense1 = tf.layers.dense(flatten,120,activation=tf.nn.relu)
    dense1 = tf.nn.dropout(dense1, keep_prob)
    dense2 = tf.layers.dense(dense1,84,activation=tf.nn.relu)
    dense3 = tf.layers.dense(dense2,43)
    return dense3

In [7]:
keep_prob = tf.placeholder(tf.float32)
logits = LeNet(features)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels_oh)
#tf.summary.histogram("cross_entropy",cross_entropy)
loss_op= tf.reduce_mean(cross_entropy)
#tf.summary.histogram("loss",loss_op)
train_op= tf.train.AdamOptimizer().minimize(loss_op)
correct_prediction = tf.equal(tf.argmax(logits,1), tf.argmax(labels_oh,1))
accuracy_op = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))


In [8]:
### Train your model here.
### Feel free to use as many code cells as needed.

BATCH_SIZE=64
def eval_on_data(X,y):
    num_examples = len(X)
    total_acc = 0
    total_loss = 0
    sess = tf.get_default_session()
    for offset in range(0,num_examples,BATCH_SIZE):
        #X,y =shuffle(X,y)
        
        end = offset + BATCH_SIZE
        X_batch = X[offset:end]
        y_batch = y[offset:end]
        loss,acc = sess.run([loss_op,accuracy_op], feed_dict ={features:X_batch,labels:y_batch,keep_prob:1.0})
        total_loss += (loss*len(X_batch))
        total_acc += (acc*len(X_batch))
    return total_loss/num_examples,total_acc/num_examples

In [ ]:
from sklearn.utils import shuffle
import time
training_epochs=50
batch_size=64
val_loss_his=[]
val_acc_his=[]

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    num_examples=len(X_train)
    for i in range(training_epochs):
        X_train,y_train =shuffle(X_train,y_train)
        t0 =time.time()
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            X_batch, y_batch = X_train[offset:end], y_train[offset:end]
            sess.run(train_op, feed_dict={features: X_batch, labels: y_batch,keep_prob:0.5})
         
        val_loss, val_acc = eval_on_data(X_val, y_val)
        val_loss_his.append(val_loss)
        val_acc_his.append(val_acc)
        print("Epoch", i+1)
        print("Time: %.3f seconds" % (time.time() - t0))
        print("Validation Loss =%.4f" %(val_loss))
        print("Validation Accuracy =%.4f" %(val_acc))
        print("")
        
        if (i+1) % 5 ==0:
            saver.save(sess, 'traffic_sign_keep0.5')
            print("Model saved")

Epoch 1
Time: 10.450 seconds
Validation Loss =2.5429
Validation Accuracy =0.2935

Epoch 2
Time: 11.096 seconds
Validation Loss =1.8726
Validation Accuracy =0.4364

Epoch 3
Time: 11.821 seconds
Validation Loss =1.6464
Validation Accuracy =0.4639

Epoch 4
Time: 10.412 seconds
Validation Loss =1.4533
Validation Accuracy =0.5341

Epoch 5
Time: 11.796 seconds
Validation Loss =1.3569
Validation Accuracy =0.5489

Model saved
Epoch 6
Time: 10.668 seconds
Validation Loss =1.2211
Validation Accuracy =0.5989

Epoch 7
Time: 11.014 seconds
Validation Loss =1.1367
Validation Accuracy =0.6336

Epoch 8
Time: 10.246 seconds
Validation Loss =1.0389
Validation Accuracy =0.6608

Epoch 9
Time: 10.190 seconds
Validation Loss =1.0024
Validation Accuracy =0.6740

Epoch 10
Time: 10.304 seconds
Validation Loss =0.8753
Validation Accuracy =0.7224

Model saved
Epoch 11
Time: 11.057 seconds
Validation Loss =0.8356
Validation Accuracy =0.7316

Epoch 12
Time: 10.607 seconds
Validation Loss =0.8174
Validation Accurac

KeyboardInterrupt: 

In [ ]:
# Test my model
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
loader = tf.train.import_meta_graph('traffic_sign.meta')
loader.restore(sess, tf.train.latest_checkpoint('./'))

test_loss, test_acc = eval_on_data(X_test, y_test)
print("Test Accuracy = {:.4f}".format(test_acc))